# 🥂 VibeCheck DC — CLIP-Based Vibe Classifier (Google Colab)
### Uses CLIP ViT-B/32 instead of YOLO
### Fetches 5 random DC restaurants via Outscraper
### Filter images for "vibe" vs "food" using CLIP semantic scores

⚠️ **Before running, replace `YOUR_KEY_HERE` with your real Outscraper API key.**

---

In [6]:
!pip install outscraper transformers pillow ftfy regex tqdm --quiet

ERROR: Operation cancelled by user


In [7]:
import json
import random
from io import BytesIO
from pathlib import Path

import requests
import torch
from outscraper import ApiClient
from PIL import Image
from transformers import CLIPModel, CLIPProcessor

# ------------------------------
# CONFIG
# ------------------------------

OUTSCRAPER_API_KEY = "NDA1NWE2OTY1YzJkNDE1MDljM2MyMDVkZGY3NGQ4MjJ8YWE3OTNhM2I4Zg"  #  <<<<<<  REPLACE ME
REVIEWS_NEEDED = 5
IMAGES_NEEDED = 5
PHOTOS_TO_FETCH = 10
DEBUG = False

OUTPUT_DIR = Path("./vibecheck_output/")
IMAGES_DIR = OUTPUT_DIR / "images"
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)
IMAGES_DIR.mkdir(exist_ok=True, parents=True)


def debug_print(x):
    if DEBUG:
        print("DEBUG:", x)


device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

ImportError: cannot import name 'GenerationMixin' from 'transformers.generation' (/usr/local/lib/python3.12/dist-packages/transformers/generation/__init__.py)

---
## 🔮 Load CLIP ViT-B/32
This version does **not** require torchvision → avoids the SYMPY error.

---

In [ ]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

text_prompts = [
    "a close-up photo of food on a plate, detailed dish photograph",
    "the interior ambience of a restaurant, dining room vibe lighting",
]

print("CLIP model loaded.")

---
## 🔎 CLIP classification helper
Returns: `(score_food, score_vibe, label)`

---

In [ ]:
def clip_classify_image(url):
    """
    Downloads an image, evaluates with CLIP, returns:
        score_food, score_vibe, label ("VIBE" or "FOOD")
    """
    try:
        img = Image.open(BytesIO(requests.get(url, timeout=10).content)).convert("RGB")
    except Exception as e:
        debug_print(f"CLIP could not load image: {e}")
        return None, None, "ERROR"

    inputs = processor(
        text=text_prompts, images=img, return_tensors="pt", padding=True
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        probs = outputs.logits_per_image.softmax(dim=1)[0]

    score_food = probs[0].item()
    score_vibe = probs[1].item()

    label = "VIBE" if score_vibe > score_food else "FOOD"
    return score_food, score_vibe, label

---
## 🍷 Filter vibe photos using CLIP
Keeps only photos where `score_vibe > score_food`.

---

In [ ]:
def filter_vibe_photos_clip(photos, needed=5):
    kept = []

    for i, p in enumerate(photos):
        if len(kept) >= needed:
            break

        url = p.get("photo_url") or p.get("original") or p.get("url")
        if not url:
            continue

        score_food, score_vibe, label = clip_classify_image(url)
        if label == "ERROR":
            print(f"  {i+1:02d}. ERROR loading image")
            continue

        print(f"  {i+1:02d}. {label}  (vibe={score_vibe:.3f}, food={score_food:.3f})")

        if label == "VIBE":
            kept.append((p, url))

    return kept

---
## 🗺️ Outscraper helper functions
Fetch restaurants, reviews, and photos.

---

In [ ]:
def get_random_dc_restaurants(client, n=5):
    print("📍 Fetching DC restaurants...")

    res = client.google_maps_search(
        "restaurants in washington dc", limit=50, language="en", region="us"
    )
    if not res:
        print("❌ No results returned.")
        return []

    places = res[0]
    names = list({p["name"] for p in places if p.get("name")})

    if len(names) < n:
        print("❌ Not enough restaurants found.")
        return []

    selected = random.sample(names, n)
    print("🎯 Selected restaurants:")
    for i, name in enumerate(selected, 1):
        print(f"  {i}. {name}")
    return selected


def get_restaurant_info(client, name):
    res = client.google_maps_search(f"{name} restaurant washington dc", limit=1)
    if not res or not res[0]:
        return None

    place = res[0][0]
    place_id = place.get("place_id") or place.get("google_id") or place.get("cid")

    return {
        "name": place.get("name"),
        "place_id": place_id,
        "rating": place.get("rating"),
        "address": place.get("full_address"),
        "type": place.get("type"),
    }


def get_reviews(client, place_id, limit=5):
    res = client.google_maps_reviews(
        [place_id], reviews_limit=limit, sort="most_relevant"
    )
    if not res:
        return []
    d = res[0]
    for key in ["reviews_data", "reviews", "data"]:
        if key in d:
            return d[key][:limit]
    return []


def get_photos(client, place_id, limit=10):
    res = client.google_maps_photos([place_id], photosLimit=limit)
    if not res:
        return []
    d = res[0]
    for key in ["photos", "photos_data", "data"]:
        if key in d:
            return d[key][:limit]
    return []

---
## 🏭 Main processing function

---

In [ ]:
def process_restaurant(client, name, index):
    print("\n" + "─" * 60)
    print(f"[{index+1}] 🍽️ {name}")
    print("─" * 60)

    info = get_restaurant_info(client, name)
    if not info:
        print("❌ Could not fetch restaurant info.")
        return None

    print(f"Found: {info['name']} ({info['rating']}⭐)")

    reviews = get_reviews(client, info["place_id"], REVIEWS_NEEDED)
    if len(reviews) < REVIEWS_NEEDED:
        print("❌ Not enough reviews.")
        return None
    print(f"✅ Got {len(reviews)} reviews")

    photos = get_photos(client, info["place_id"], PHOTOS_TO_FETCH)
    if not photos:
        print("❌ No photos found.")
        return None

    print(f"📷 Found {len(photos)} photos — filtering via CLIP...")
    vibe_photos = filter_vibe_photos_clip(photos, needed=IMAGES_NEEDED)

    if len(vibe_photos) < IMAGES_NEEDED:
        print("❌ Not enough vibe photos.")
        return None

    print(f"🎉 Kept {len(vibe_photos)} vibe photos")

    saved_files = []
    safe = "".join(c if c.isalnum() else "_" for c in info["name"])

    for idx, (_photo_obj, url) in enumerate(vibe_photos):
        filename = f"{safe}_{idx+1}.jpg"
        filepath = IMAGES_DIR / filename
        try:
            img_bytes = requests.get(url, timeout=10).content
            with open(filepath, "wb") as f:
                f.write(img_bytes)
            saved_files.append(filename)
        except (requests.RequestException, OSError):
            continue

    return {
        "restaurant": info,
        "reviews": reviews,
        "photos": saved_files,
    }

---
# ▶️ Run VibeCheck for 5 restaurants

---

In [ ]:
client = ApiClient(api_key=OUTSCRAPER_API_KEY)

restaurants = get_random_dc_restaurants(client, n=5)
results = []

for i, name in enumerate(restaurants):
    out = process_restaurant(client, name, i)
    if out:
        results.append(out)

with open(OUTPUT_DIR / "results.json", "w") as f:
    json.dump(results, f, indent=2)

print("\n🎉 DONE! Saved results and images to:", OUTPUT_DIR)

---
## 📥 Download results (Colab only)
Run this cell to zip and download your results.

---

In [ ]:
# Optional: Download results from Colab
import shutil

from google.colab import files

shutil.make_archive("vibecheck_results", "zip", OUTPUT_DIR)
files.download("vibecheck_results.zip")